Genetic Algorithm - An algorithm that pits different solutions against each other and uses selection to "evolve" them into the best solution.

Genetic algorithms efficiently explore the solution space to discover optimal or near-optimal solutions.

By mimicking the process of natural selection, GAs can evolve solutions over time, improving their performance iteratively. 

Genetic Algorithms were inspired by evolution. Consequently, the components of a GA share names and functions similar to those of their biological counterparts. 

---

**Key Terminology**

**Chromosomes** - potential solutions to a problem, typically coded as strings or arrays. Each element in a chromosome corresponds to a gene that determines a specific trait of the solution. 

**Crossover** - involves exchanging segments of the *'parent'* chromosomes to create new *'individuals'*. This operation introduces variability and allows the *'offspring'* to inherit beneficial traits from both *'parents'*.

**Mutation** - randomly altering one or more genes. 

---

**Components of a Genetic Algorithm**

**Population** - a set of candidate solutions, often called individuals. Each individual is represented as a chromosome, which can be coded as binary strings, arrays, or other data structures. 

**Fitness Function** - evaluates each individual's ability to solve the problem we're interested in. It assigns a fitness value to each individual, with higher values for better solutions. The fitness function guides the algorithm toward better solutions. 

**Selection Function** - chooses individuals from the population to reproduce based on their fitness. Individuals with a higher fitness are more likely to be chosen for reproduction. There are a few different selection methods:
- **Roulette Wheel Selection** is where individuals are selected based on a probability proportional to their fitness, similar to spinning a roulette wheel. 
- **Tournament Selection** is where a subset of individuals is chosen at random and the fittest individual from this subset is selected.
- **Rank-based Selection** ranks individuals based on their fitness, then selection probabilities are assigned based on these ranks rather than raw fitness values. 

**Crossover Function** - combines information from two individuals to create offspring; the goal is to inherit beneficial traits from both parents. Common crossover techniques include:
- **Single-point Crossover** which involves choosing a random point and swapping the genetic material from two parents at that point to create two offspring. 
- **Multi-point Crossover** uses multiple points for swapping segments between parents, allowing for more complex genetic combinations.
- **Uniform Crossover** randomly decides which parent will contribute each gene, providing a high level of genetic diversity.
- **Blend Crossover** generates offspring by blending the genes of the parents using a random blending factor.

**Mutation Function** - mutations introduce random changes in the offspring's genetic material. This helps maintain diversity in the population and explores new areas of the solution space. 

---

**The Genetic Algorithm Process**

A genetic algorithm goes through a series of steps that mimic natural evolutionary processes to find optimal solutions. These steps allow the population to evolve over generations, improving the quality of solutions. This is generally how a genetic algorithm proceeds:

**Step 1: Initialization**
First generate an initial population of random individuals. This step creates a diverse set of potential solutions to start the algorithm. 

**Step 2: Evaluation**
Next calculate the fitness of each individual in the population. Here we use the fitness function to evaluate how good each solution is.

**Step 3: Selection**
Using the selection criteria, we select individuals for reproduction based on their fitness. This step determines which individuals will be parents.

**Step 4: Crossover**
Crossover is done by combining the genetic material of selected parents, we apply crossover techniques to generate new solutions or offspring.

**Step 5: Mutation**
To maintain diversity in our population, we need to introduce random mutations in the offspring.

**Step 6: Replacement**
We replace some or all of the old population with the new offspring, by determining which individuals move on to the next generation.

**Step 7: Repeat**
The previous steps 2-6 are looped over for a set number of generations or until a termination condition is met. This loop allows the population to evolve over time, hopefully resulting in a good solution.

---

Imports

In [1]:
import random   # Generate random values for starting population
import matplotlib.pyplot as plt
import numpy as np
from prettytable import PrettyTable # Used for visualizations

Define Fitness Function

In [2]:
def fitness_function(params):
    a, b, c = params
    if a < 0:
        return -float('inf')    # Penalize downward facing u-shapes heavily
    vertex_x = -b / (2 * a)     # x value at vertex
    vertex_y = a * (vertex_x ** 2) + b * vertex_x + c  # y value at vertex
    y_left = a * (-1) ** 2 + b * (-1) + c   # y-coordinate at x = -1
    y_right = a * (1) ** 2 + b * (1) + c    # y-coordinate at x = 1
    curviness = abs(y_left - vertex_y) + abs(y_right = vertex_y)
    return -curviness   # Negate to minimize curviness

Create initial population

In [3]:
def create_initial_population(size, lower_bound, upper_bound):
    population = []
    for _ in range(size):
        individual = (random.uniform(lower_bound, upper_bound),
                      random.uniform(lower_bound, upper_bound),
                      random.uniform(lower_bound, upper_bound))
        population.append(individual)
    return population

Create a selection function

In [4]:
def selection(population, fitnesses, tournament_size=3):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(list(zip(population, fitnesses)), tournament_size)
        winner = max(tournament, key=lambda x: x[1])[0]
        selected.append(winner)
    return selected

Make a Crossover function

In [5]:
def crossover(parent1, parent2):
    alpha = random.random()
    child1 = tuple(alpha * p1 + (1-alpha) * p2 for p1, p2 in zip(parent1, parent2))
    child2 = tuple(alpha * p1 + (1-alpha) * p2 for p1, p2 in zip(parent1, parent2))
    return child1, child2

Design a mutation function

In [6]:
def mutation(individual, mutation_rate, lower_bound, upper_bound):
    individual = list(individual)
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            mutation_amount = random.uniform(-1, 1)
            individual[i] += mutation_amount
            # Ensure the individual stays within upper bounds
            individual[i] = max(min(individual[i], upper_bound), lower_bound)
    return tuple(individual)